# Libs

In [2]:
import numpy as np

# Tab Data
import pandas as pd

# Geo Data
import geopandas as gpd
from shapely.geometry import shape

# Plotting
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

In [3]:
import local_functions.features as f
import local_functions.preprocessing as p

# Catalogs

In [4]:
DAYS = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

# Functinos

In [5]:
def top_10_PUlocations(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Retrieves the top 10 most frequent pickup locations based on the specified column.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data.
        col (str): The column name to use for counting pickup locations.

    Returns:
        pd.DataFrame: A DataFrame containing the top 10 pickup locations with their counts and ranks.
    """
    df = (
        df.value_counts(col)
        .head(10)
        .reset_index()
        .reset_index()
        .rename(columns={"index": "rank"})
        .assign(rank=lambda x: x["rank"] + 1)
    )

    return df

In [6]:
def nyc_heatmap(
    df: pd.DataFrame,
    geo_df: pd.DataFrame,
    feature: str,
    title: str,
    frames: int,
    days: dict[int, str] = None,
  ) -> None:
    """
    Creates an animated heatmap visualization for NYC locations based on the ranking of a specified feature.

    Args:
        df (pd.DataFrame): The input DataFrame containing ranking data for NYC locations.
        geo_df (pd.DataFrame): The GeoDataFrame containing geographical data for NYC locations.
        feature (str): The feature by which the locations will be ranked for each frame of the animation.
        title (str): The title of the heatmap animation.
        frames (int): The number of frames for the animation (e.g., hours or days).
        days (dict[int, str], optional): A mapping of frame indices to day names (if applicable).

    Returns:
        None: The function displays an HTML5 video of the animation and does not return any value.
    """

    def drawframe(frame: int) -> tuple[plt.Axes, plt.Axes]:
        """
        Draws a single frame of the heatmap animation, visualizing NYC location rankings
        based on the specified feature for a given frame (e.g., hour or day).

        Args:
            frame (int): The current frame index used to query and visualize the data.

        Returns:
            tuple[plt.Axes, plt.Axes]: A tuple of matplotlib axes for the two subplots created in the frame.
        """
        global cbar_added  # Use global to modify the cbar_added flag

        # Extract the ranking data for the specific hour
        location_rank = (
            df.query(f"{feature} == {frame}")
            .set_index("pu_location_id")["rank"]
            .to_dict()
        )

        # Update the 'color' column based on the rank
        geo_df["color"] = geo_df["LocationID"].apply(
            lambda x: location_rank[x] if x in location_rank.keys() else 0
        )

        # Clear the previous plots
        ax1.clear()
        ax2.clear()

        # Create the two subplots
        geo_df.query("color != 0").plot(
            column="color", legend=False, cmap="OrRd_r", ax=ax1
        )
        geo_df.query("color == 0").plot(column="color", legend=False, ax=ax1)

        geo_df.query("color != 0").plot(
            column="color",
            legend=not cbar_added,  # Only add the legend (color bar) if it hasn't been added yet
            legend_kwds={"shrink": 0.5},
            cmap="OrRd_r",
            ax=ax2,
        )
        geo_df.query("color == 0").plot(column="color", legend=False, ax=ax2)

        ax2.axis([0.971e6, 1.01e6, 180_000, 230_000])

        # Only add the color bar on the first frame
        if not cbar_added and ax2.get_figure().axes[-1] is not None:
            cbar = ax2.get_figure().axes[-1]
            cbar.invert_yaxis()
            cbar_added = True  # Set the flag to True to prevent future additions

        # Remove ticks for a cleaner look
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax2.set_xticks([])
        ax2.set_yticks([])

        # Set the title for the figure
        if days != None:
            frame = days[frame]
        fig.suptitle(f"{title}: {frame}")

        return ax1, ax2

    # Create figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Create the animation using FuncAnimation
    anim = animation.FuncAnimation(
        fig, drawframe, frames=frames, interval=500, blit=False  # 24 hours animation
    )

    # Display the animation

    display(HTML(anim.to_html5_video()))
    plt.close()

# Data

## Zones Lookup

In [7]:
lookup = pd.read_csv("./data/taxi_zones_lookup.csv")
lookup_map = lookup.set_index("LocationID")["Zone"].to_dict()
lookup_map[264] = "Unknown"  # Fix missing value

In [8]:
lookup.head()

LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone

## Geo Data

In [9]:
geo_df = gpd.read_file("./data/taxi_zones_shapefile/taxi_zones.shp")
geo_df = p.preprocess_geo_data(geo_df)
borough_map, area_map = p.get_geo_maps(geo_df)
geo_df.head()

OBJECTID  Shape_Leng  Shape_Area                         zone  \
22         23    0.290556    0.002196      Bloomfield/Emerson Hill   
131       132    0.245479    0.002038                  JFK Airport   
117       118    0.243966    0.001827  Heartland Village/Todt Hill   
43         44    0.235689    0.001945       Charleston/Tottenville   
1           2    0.433470    0.004866                  Jamaica Bay   

     LocationID        borough  \
22           23  Staten Island   
131         132         Queens   
117         118  Staten Island   
43           44  Staten Island   
1             2         Queens   

                                              geometry  area_mi2  \
22   POLYGON ((929937.155 170690.809, 929967.456 17...  7.586531   
131  MULTIPOLYGON (((1032791.001 181085.006, 103283...  7.032865   
117  POLYGON ((954167.794 161619.974, 954322.65 161...  6.630996   
43   POLYGON ((925284.273 142075.259, 924921.951 14...  5.230083   
1    MULTIPOLYGON (((1033269.244 172126.008, 103343...  5.161748   

                           centroid  
22    POINT (936800.381 160268.124)  
131  POINT (1043488.187 175062.678)  
117     POINT (947313.212 153002.2)  
43    POINT (919325.671 130813.717)  
1    POINT (1031085.719 164018.754)

## Trip Data

In [10]:
trip_features = [
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
]

drop_features = ["store_and_fwd_flag"]

LOW_QUANTILE = 0.01
HIGH_QUANTILE = 0.95

us_holidays = p.get_us_holidays(2023)

In [11]:
df = pd.read_csv(
    "./data/yellow_taxi_trip_records_2023.csv",
)

<ipython-input-11-976cf4621e8a>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [12]:
df = (
    df.drop(columns=drop_features)
    .pipe(p.normalize_columns)
    .pipe(p.col_to_datetime)
    .pipe(p.valid_vendor_payment_filter)
    .pipe(p.valid_trip_filter, cols=trip_features)
    .pipe(p.quantile_filter, cols=trip_features, low=LOW_QUANTILE, high=HIGH_QUANTILE)
    .pipe(p.impute_missing)
    .pipe(f.calculate_time_features, us_holidays)
    .pipe(f.calculate_trip_features)
)

df = df.assign(
    pu_name=df.pu_location_id.map(lookup_map),
    do_name=df.do_location_id.map(lookup_map),
)

# Questions

## 1.  What was the average speed of taxi trips per day of the week?

In [13]:
def q1(df: pd.DataFrame, days: dict[int, str]) -> pd.DataFrame:
    """
    Aggregates the average speed (in kilometers and miles per hour) of trips
    by the day of the week for the input DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup weekdays.
        days (dict[int, str]): A mapping of weekday numbers to their corresponding names.

    Returns:
        pd.DataFrame: A DataFrame with the average speed calculated for each weekday,
                      including a mapped weekday name.
    """

    df = (
        df.groupby(["tpep_pickup_weekday"])
        .agg(
            avg_speed_kmh=("speed_kmh", "mean"),
            avg_speed_mph=("speed_mph", "mean"),
        )
        .reset_index()
    )

    df = df.assign(Weekday=df.tpep_pickup_weekday.map(days)).reindex(
        columns=["Weekday", "tpep_pickup_weekday", "avg_speed_kmh", "avg_speed_mph"]
    )

    return df

In [14]:
q1_df = q1(df, DAYS)
q1_df

Weekday  tpep_pickup_weekday  avg_speed_kmh  avg_speed_mph
0     Monday                    0      16.619418      10.329035
1    Tuesday                    1      15.238663       9.470891
2  Wednesday                    2      15.010209       9.328906
3   Thursday                    3      14.926466       9.276859
4     Friday                    4      15.541483       9.659094
5   Saturday                    5      16.631881      10.336781
6     Sunday                    6      18.376039      11.420782

In [15]:
fig = px.line(q1_df, x="tpep_pickup_weekday", y="avg_speed_kmh", markers=True)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q1_df.tpep_pickup_weekday.values,
        ticktext=q1_df.Weekday.values,
    ),
    title="Avrage speed per day of the week",
    yaxis_title="km/h",
    xaxis_title=None,
)
fig.show()

The plot presents the average speed of taxi trips throughout the week, revealing interesting patterns in urban mobility. Thursday shows the lowest average speed at approximately 14.93 km/h (9.28 mph), which may indicate higher traffic congestion as both tourists and local workers are commuting to their destinations. This trend suggests that the midweek period is particularly busy, leading to slower travel times.

In contrast, Sunday records the highest average speed at 18.38 km/h (11.42 mph), likely due to a reduced number of commuters and less traffic overall. The weekend, particularly Sunday, seems to facilitate smoother travel as people engage in leisure activities rather than daily commutes. Overall, this analysis emphasizes how varying traffic patterns across the week impact taxi speeds, highlighting the importance of understanding these dynamics for effective transportation planning.

## 2. What was the average fare_amount of taxi trips per hour of the day?

In [16]:
def q2(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average fare amount for trips grouped by pickup hour.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with fare amounts.

    Returns:
        pd.DataFrame: A DataFrame with the average fare amount for each pickup hour.
    """
    return (
        df.groupby(["tpep_pickup_hour"])
        .agg(avg_fare_amount=("fare_amount", "mean"))
        .reset_index()
    )

In [17]:
q2_df = q2(df)

In [18]:
fig = px.line(q2_df, x="tpep_pickup_hour", y="avg_fare_amount", markers=True)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q2_df.tpep_pickup_hour.values,
    ),
    title="Avrage fare by pickup hour",
    yaxis_title="USD",
    xaxis_title=None,
)
fig.show()

The data on average taxi fare amounts throughout the day reveals interesting trends in pricing and demand. The highest average fare occurs at 11 PM, with a fare amount of $\$$ 14.60, while the lowest average fare is recorded at 6 AM, with just $\$$12.29. This pattern suggests that late-night rides may involve longer distances or be driven by higher demand for taxis after events, parties, or social outings, which can lead to increased fares.

In the early morning hours, the lower average fares could be indicative of shorter trips as people tend to travel to their workplaces or early morning activities. Overall, the data highlights the dynamic nature of taxi fares, influenced by the time of day, passenger demand, and the types of trips being taken. Understanding these fluctuations can help taxi companies and drivers optimize their services and pricing strategies to better meet customer needs throughout the day.

## 3. How many trips did taxis take per day of the week?

In [19]:
def q3(df: pd.DataFrame, days: dict[int, str]) -> pd.DataFrame:
    """
    Counts the number of trips for each day of the week and maps the
    weekday numbers to their corresponding names.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup weekdays.
        days (dict[int, str]): A mapping of weekday numbers to their corresponding names.

    Returns:
        pd.DataFrame: A DataFrame with the count of trips for each weekday,
                      including a mapped weekday name.
    """
    df = (
        df.value_counts(["tpep_pickup_weekday"])
        .reset_index()
        .sort_values("tpep_pickup_weekday")
    )

    df = (
        df.assign(Weekday=df.tpep_pickup_weekday.map(days))
        .rename(columns={"count": "trip_count"})
        .reindex(columns=["Weekday", "tpep_pickup_weekday", "trip_count"])
    )
    return df

In [20]:
q3_df = q3(df, DAYS)
q3_df

Weekday  tpep_pickup_weekday  trip_count
5     Monday                    0      641878
2    Tuesday                    1      791714
1  Wednesday                    2      843559
0   Thursday                    3      854580
3     Friday                    4      784993
4   Saturday                    5      780408
6     Sunday                    6      631267

In [21]:
fig = px.bar(
    q3_df,
    x="tpep_pickup_weekday",
    y="trip_count",
)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q3_df.tpep_pickup_weekday.values,
        ticktext=q3_df.Weekday.values,
    ),
    title="Totla trips taken by weekday",
    yaxis_title="Trips taken",
    xaxis_title=None,
)
fig.show()

The data on taxi trip counts throughout the week highlights significant fluctuations in demand. Wednesday has the highest number of trips, with a total of 843,559, followed closely by Thursday at 854,580 trips. This spike in midweek activity suggests that these days are particularly busy for taxi services, possibly due to a combination of commuter traffic and local events.

In contrast, Sunday records the lowest trip count at 631,267. This decrease may reflect a general decline in commuter travel as many people engage in leisure activities or rest at home. Overall, the data underscores the importance of understanding daily and weekly patterns in taxi usage, which can help taxi operators manage their resources effectively to meet demand, particularly during peak days.

## 4. Find the trip starting taxi-zone with the highest tip_amount per minute?

In [22]:
df.groupby(["pu_name"]).agg(
    avg_tips_per_min=("tips_per_min", "mean")
).sort_values("avg_tips_per_min", ascending=False).reset_index().head(5)

pu_name  avg_tips_per_min
0              Jackson Heights          1.491844
1               Outside of NYC          0.975634
2  Heartland Village/Todt Hill          0.765644
3                College Point          0.710970
4                   Whitestone          0.638591

The data identifies Jackson Heights as the taxi zone with the highest average tip amount per minute, recording an impressive $1.49 in tips. This indicates that passengers in this area are likely to show greater appreciation for their drivers, possibly due to high-quality service or positive travel experiences.

In contrast, the other zones, such as Outside of NYC and Heartland Village/Todt Hill, also demonstrate noteworthy average tip amounts but fall significantly behind Jackson Heights, at $\$$0.98 and $\$$0.77, respectively. The higher tips in Jackson Heights may reflect a vibrant local community or a demographic that values good service. Overall, this analysis highlights the importance of location in determining tipping behavior, suggesting that taxi drivers operating in Jackson Heights may benefit from a more generous passenger base. Understanding these trends can help drivers and taxi companies tailor their services to enhance customer satisfaction and potentially increase tips.

## 5. What are the top 10 locations with highest demand for each hour of the day?

In [23]:
def q5(df: pd.DataFrame) -> pd.DataFrame:
    """
    Retrieves the top 10 pickup locations for each pickup hour in the input DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup locations.

    Returns:
        pd.DataFrame: A DataFrame listing the top 10 pickup locations for each pickup hour.
    """

    df = (
        df.groupby(["tpep_pickup_hour"])
        .apply(top_10_PUlocations, col="pu_location_id")
        .reset_index()
        .drop(columns="level_1")
    )
    return df

In [24]:
q5_df = q5(df)

In [25]:
cbar_added = False
nyc_heatmap(
    q5_df,
    geo_df,
    feature="tpep_pickup_hour",
    title="Taxi demand on top 10 locations by hour",
    frames=24,
)

The video illustrates the shifting demand for the top 10 locations in a city over a 24-hour period. In the early morning, demand is highest in the outskirts, indicating that workers are commuting to the city for their jobs. As the day progresses, particularly from mid-morning to early afternoon, the focus shifts to the city center, which becomes a hub for commerce, dining, and entertainment, attracting both employees and visitors.

In the evening, demand fluctuates again, with areas near nightlife options rising in popularity as people seek social activities. This trend highlights the city's dynamic nature, where daily routines and social patterns significantly influence mobility.

In [26]:
q5_df["pu_name"] = q5_df.pu_location_id.map(lookup_map)

fig = px.line(
    q5_df.query("rank < 6"),
    x="tpep_pickup_hour",
    y="rank",
    color="pu_name",
    markers=True,
)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q5_df.tpep_pickup_hour.values,
        ticktext=q5_df.tpep_pickup_hour.values,
    ),
    yaxis=dict(
        autorange="reversed",
        tickvals=q5_df["rank"].unique(),
    ),
    title="Top 5 locatons per hour of day",
    yaxis_title="Rank",
    xaxis_title=None,
)
fig.show()

If we plot the top 5 locations by demand we can see there is no location where demand is constant during the day.

## 6. What are the top 10 locations with highest demand for each day of the week?

In [27]:
def q6(df: pd.DataFrame, days: dict[int, str]) -> pd.DataFrame:
    """
    Retrieves the top 10 pickup locations for each day of the week in the input DataFrame
    and maps the weekday numbers to their corresponding names.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup locations.
        days (dict[int, str]): A mapping of weekday numbers to their corresponding names.

    Returns:
        pd.DataFrame: A DataFrame listing the top 10 pickup locations for each weekday,
                      including a mapped weekday name.
    """

    df = (
        df.groupby(["tpep_pickup_weekday"])
        .apply(top_10_PUlocations, col="pu_location_id")
        .reset_index()
        .drop(columns="level_1")
    )

    df = df.assign(
        Weekday=df.tpep_pickup_weekday.map(days),
    ).reindex(
        columns=["Weekday", "tpep_pickup_weekday", "rank", "pu_location_id", "count"]
    )

    return df

In [28]:
q6_df = q6(df, DAYS)

In [29]:
cbar_added = False
nyc_heatmap(
    q6_df,
    geo_df,
    feature="tpep_pickup_weekday",
    title="Taxi demand on top 10 locations by weekday",
    frames=7,
    days=DAYS,
)

The video depicts the variations in demand for different locations throughout the week, highlighting a consistent pattern on workdays compared to weekends. During weekdays, certain areas, particularly those in business districts, experience steady demand as workers commute to their jobs. This stability underscores the role of these locations as essential hubs for employment and productivity throughout the workweek.

In contrast, the weekend brings a noticeable shift in demand, with the city center emerging as the focal point for social activities and leisure pursuits. As people seek dining, entertainment, and shopping experiences, demand increases in these areas. This transition from work-oriented locations to vibrant social hubs illustrates the changing priorities of the city's population over the course of the week, emphasizing the importance of accommodating diverse needs through effective planning and resource allocation.

In [30]:
q6_df["pu_name"] = q6_df.pu_location_id.map(lookup_map)

fig = px.line(
    q6_df.query("rank < 6"),
    x="tpep_pickup_weekday",
    y="rank",
    color="pu_name",
    markers=True,
)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q6_df.tpep_pickup_weekday.values,
        ticktext=q6_df.Weekday.values,
    ),
    yaxis=dict(
        autorange="reversed",
        tickvals=q6_df["rank"].unique(),
    ),
    title="Top 5 locatons per day of the week",
    yaxis_title="Rank",
    xaxis_title=None,
)
fig.show()

In thes case and in contrast with Q5, We can see how during weekdays the top 5 bearly changes, but for the weekend top locations change.

## 7. Find the top 10 locations with the highest demand per square mile?

In [31]:
def q7(
    df: pd.DataFrame, area_map: dict[int, float], lookup_map: dict[int, str]
) -> pd.DataFrame:
    """
    Analyzes the demand for pickup locations by calculating the demand per area
    and ranking the locations based on that demand.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup location IDs.
        area_map (dict[int, float]): A mapping of pickup location IDs to their respective area sizes.
        lookup_map (dict[int, str]): A mapping of drop-off location IDs to their respective names.

    Returns:
        pd.DataFrame: A DataFrame listing pickup locations ranked by demand per area,
                      including the area size and demand per area.
    """
    df = (
        (
            df.query("pu_location_id != 264")
            .value_counts("pu_location_id")
            .reset_index()
            .assign(
                area_mi2=lambda x: x["pu_location_id"].map(area_map),
                demand_per_area=lambda x: x["count"] / x["area_mi2"],
            )
        )
        .sort_values("demand_per_area", ascending=False)
        .reset_index(drop=True)
        .reset_index()
        .rename(columns={"index": "rank"})
    )
    df = df.assign(
        rank=lambda x: x["rank"] + 1,
        pu_name=df.pu_location_id.map(lookup_map),
    )
    return df

In [32]:
df_q7 = q7(df, area_map, lookup_map)
df_q7 = df_q7.head(10)
df_q7

rank  pu_location_id   count  area_mi2  demand_per_area  \
0     1             186  200760  0.134235     1.495588e+06   
1     2             162  216992  0.173394     1.251439e+06   
2     3             163  169720  0.147900     1.147529e+06   
3     4             161  273373  0.260348     1.050028e+06   
4     5             237  293640  0.347450     8.451285e+05   
5     6             230  168791  0.203064     8.312222e+05   
6     7             142  203263  0.273827     7.423034e+05   
7     8             236  262619  0.372278     7.054382e+05   
8     9             164  134636  0.201433     6.683896e+05   
9    10             170  177684  0.269085     6.603256e+05   

                        pu_name  
0  Penn Station/Madison Sq West  
1                  Midtown East  
2                 Midtown North  
3                Midtown Center  
4         Upper East Side South  
5     Times Sq/Theatre District  
6           Lincoln Square East  
7         Upper East Side North  
8                 Midtown South  
9                   Murray Hill

In [33]:
fig = px.bar(
    df_q7,
    x="rank",
    y="demand_per_area",
)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=df_q7["rank"].values,
        ticktext=df_q7.pu_name.values,
    ),
    title="Top 10 highes demand based on zone area",
    yaxis_title="Trips taken",
    xaxis_title=None,
)
fig.show()

The data reveals the top 10 locations with the highest demand per square mile, showcasing significant urban hotspots. Notably, Penn Station/Madison Sq West ranks highest, reflecting its role as a major transportation hub that attracts both commuters and tourists. Following closely are areas like Midtown East and Midtown North, which are likely influenced by a concentration of businesses, restaurants, and attractions that drive foot traffic throughout the day.

As we move down the list, the Upper East Side and Times Square/Theatre District also emerge as key locations with substantial demand, highlighting their popularity for both local residents and visitors. The high demand in these areas may be attributed to their vibrant social scenes, cultural institutions, and retail offerings. Overall, the data underscores the importance of strategic urban planning in these high-demand areas to manage congestion and enhance accessibility, ultimately supporting the diverse needs of the city's population.

## 8. What are the top 10 locations people take taxis to get to?

In [34]:
def q8(df: pd.DataFrame, lookup_map: dict[int, str]) -> pd.DataFrame:
    """
    Analyzes the distribution of trips by drop-off location and calculates the percentage
    of total trips for each location, returning the top 10 drop-off locations.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with drop-off location IDs.
        lookup_map (dict[int, str]): A mapping of drop-off location IDs to their respective names.

    Returns:
        pd.DataFrame: A DataFrame listing the top 10 drop-off locations ranked by trip count,
                      including the percentage of total trips for each location and their names.
    """

    all_trips = len(df)

    df = (
        df.value_counts("do_location_id")
        .reset_index()
        .reset_index()
        .rename(columns={"index": "rank"})
        .assign(
            rank=lambda x: x["rank"] + 1,
            porcentage_trips=lambda x: round((x["count"] / all_trips) * 100, 2),
        )
        .head(10)
    )

    df = df.assign(
        do_name=df.do_location_id.map(lookup_map),
    )
    return df

In [35]:
q8_df = q8(df, lookup_map)

In [36]:
fig = px.bar(
    q8_df,
    x="rank",
    y="porcentage_trips",
)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=q8_df["rank"].values,
        ticktext=q8_df.do_name.values,
    ),
    title="Top 10 Destinations",
    yaxis_title="Porcentaeg of all trips (%)",
    xaxis_title=None,
)
fig.show()

The data presents the top 10 locations where people frequently take taxis, revealing key destinations within the city. Upper East Side North tops the list, accounting for 5.22% of taxi trips, closely followed by Upper East Side South at 4.91%. This suggests that the Upper East Side is a highly sought-after area, likely due to its residential appeal, cultural institutions, and proximity to Central Park.

Midtown Center and Murray Hill also feature prominently, indicating their significance as business and social hubs that attract both residents and visitors. Additionally, locations like Upper West Side South and Lincoln Square East reflect the popularity of these neighborhoods, which are known for their vibrant atmosphere and amenities. The consistent demand for taxis to these areas underscores the need for efficient transportation options that cater to the diverse needs of the city's inhabitants, from daily commuters to those seeking leisure activities. Overall, this data highlights the interconnectedness of these neighborhoods and their roles as key destinations in the urban landscape.

## 9. What hour of the day is the busiest hour?

In [37]:
def q9(df: pd.DataFrame) -> pd.DataFrame:
    """
    Counts the number of trips for each pickup hour in the input DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup hours.

    Returns:
        pd.DataFrame: A DataFrame listing the count of trips for each pickup hour,
                      sorted by the hour.
    """

    df = (
        df.value_counts("tpep_pickup_hour", normalize=True)
        .reset_index()
        .sort_values("tpep_pickup_hour")
    )
    df = df.assign(proportion=round(df.proportion * 100, 2))
    return df

In [38]:
df_q9 = q9(df)

In [39]:
fig = px.line(df_q9, x="tpep_pickup_hour", y="proportion", markers=True)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=df_q9.tpep_pickup_hour.values,
    ),
    title="Proportion of trips taken by hour of day",
    yaxis_title="Porcentage (%)",
    xaxis_title=None,
)
fig.show()

Wher we can see the busiest hour for taxi pickups, revealing that the peak demand occurs at 6 PM, with a proportion of 7.54%. This high volume likely reflects the end of the workday when many commuters are leaving offices and heading home or to social engagements.

The analysis shows a gradual increase in taxi usage throughout the late afternoon and early evening, peaking at 6 PM before starting to decline. Notably, the hours from 5 PM to 8 PM consistently exhibit high demand, indicating that this time frame is critical for transportation services. The patterns suggest that urban mobility is heavily influenced by work schedules and social activities, emphasizing the need for efficient taxi services during these peak hours to meet the demands of city dwellers.

## 10. What is the average taxi trip duration per hour of the day?

In [40]:
def q10(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average trip duration (in minutes) for trips grouped by pickup hour.

    Args:
        df (pd.DataFrame): The input DataFrame containing trip data with pickup hours and trip durations.

    Returns:
        pd.DataFrame: A DataFrame with the average trip duration for each pickup hour.
    """

    return (
        df.groupby(["tpep_pickup_hour"])
        .agg(avg_trip_duration_min=("trip_distance_mi", "mean"))
        .reset_index()
    )

In [41]:
df_q10 = q10(df)

In [42]:
fig = px.line(df_q10, x="tpep_pickup_hour", y="avg_trip_duration_min", markers=True)
fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=df_q10.tpep_pickup_hour.values,
    ),
    title="Avrage trip duration by hour of day",
    yaxis_title="Min",
    xaxis_title=None,
)
fig.show()

The plot illustrates the average taxi trip duration across different hours of the day, revealing notable trends in travel time. The shortest average trip durations occur in the morning, particularly between 8 AM and 11 AM, with the lowest average duration recorded at 1.87 minutes around 10 AM. This may suggest that taxi trips during these hours are typically shorter, possibly due to the concentration of people commuting within more compact areas.

Conversely, average trip durations begin to increase during the afternoon and evening hours, peaking at around 2.68 minutes at 4 AM. This trend might indicate that as the day progresses, taxi trips become longer due to increased traffic congestion and longer distances traveled, particularly as people start their nightlife activities. The data highlights how urban dynamics influence travel times, emphasizing the importance of considering traffic patterns when planning transportation services.